# VCP演習

以下の演習課題から選択してください。

1. VCノードを利用してLaTeX文書をPDFファイルに変換してみよう
2. VCノードを利用してWordPress（またはownCloud）環境を構築してみよう

## VCノードを利用してLaTeX文書をPDFファイルに変換してみよう

ここでは、[301-AWSスポットインスタンスの利用](./301-AWSスポットインスタンスの利用.ipynb) などと同様にVCノードを作成し、  
LaTeXで書かれた文書をPDFに変換するための処理をアプリケーションコンテナ上で実行します。

### AWSにVCノードを作成

- 任意のインスタンスタイプを利用（通常は `'small'` で十分）
- SSH公開鍵を設定する（起動後にSSHログインが必要なため）

In [ ]:
# アクセストークン入力
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
# VCP SDK 初期化
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

In [ ]:
# UnitGroup 作成
unit_group = vcp.create_ugroup('handson401')

In [ ]:
# spec 設定 (スポットインスタンスの場合は aws_spot を指定)
spec = vcp.get_spec('aws', 'small')

In [ ]:
# ssh公開鍵設定
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')
spec.set_ssh_pubkey(ssh_public_key)

In [ ]:
# spec内容確認
print(spec)

In [ ]:
# Unit 作成（VCノード起動）
unit = unit_group.create_unit('aws-server', spec)

In [ ]:
# VCノード情報確認
unit_group.df_nodes()

In [ ]:
# VCノードのIPアドレスを変数に格納
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返す。
# 起動しているVCノードが1つの場合、先頭要素 [0] を取得する。
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0]
print(ip_address)

### VCノードにSSHログイン

NotebookのTerminalを開いてsshコマンドを実行してログインします。
```
ssh root@{VCノードIPアドレス}
```

あるいは、[301-AWSスポットインスタンスの利用](./301-AWSスポットインスタンスの利用.ipynb) と同様にNotebookのセルにsshコマンドを記述して実行する方法もあります。

### LaTeXサンプルファイルを入手

電子情報通信学会 LaTeXスタイルファイル http://www.ieice.org/ftp/ をサンプルとして利用します。

VCノード上で wget コマンドを使ってサンプルファイルをダウンロードします。
```
wget http://www.ieice.org/ftp/tex/ieicej/LaTeX2e/ieicej3.2.zip
```

In [ ]:
# VCノードのホストキー取得 (Notebook上での対話処理を避けるための準備)
!touch ~/.ssh/known_hosts
!ssh-keygen -R {ip_address}
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts

In [ ]:
# VCノード上で wget コマンド実行
!ssh root@{ip_address} wget http://www.ieice.org/ftp/tex/ieicej/LaTeX2e/ieicej3.2.zip

### LaTeX環境インストール済みコンテナを実行

VCノード上で以下のコマンドを実行します。

```bash
# 入手したアーカイブを展開し、texファイルがあるフォルダに移動
unzip ieicej3.2.zip
cd ieicej3.2/UTF

# uplatex にて tex → dvi 変換
docker run --rm -v ${PWD}:/workdir paperist/alpine-texlive-ja uplatex readme.tex

# dvipdfmx にて dvi → pdf 変換
docker run --rm -v ${PWD}:/workdir paperist/alpine-texlive-ja dvipdfmx readme.dvi
```

In [ ]:
# VCノード上で unzip コマンド実行
!ssh root@{ip_address} unzip ieicej3.2.zip

In [ ]:
# VCノード上で uplatex コマンドをコンテナ実行 (tex → dvi 変換)
!ssh root@{ip_address} \
  'cd ieicej3.2/UTF && /usr/local/bin/docker run --rm -v $(pwd):/workdir paperist/alpine-texlive-ja uplatex readme.tex'

In [ ]:
# VCノード上で dvipdfmx コマンドをコンテナ実行 (dvi → pdf 変換)
!ssh root@{ip_address} \
  'cd ieicej3.2/UTF && /usr/local/bin/docker run --rm -v $(pwd):/workdir paperist/alpine-texlive-ja dvipdfmx readme.dvi'

### 生成されたPDFファイルをNotebook環境にコピー (scp)

- VCノードからログアウトし、NotebookのTeminalからscpコマンドを使ってVCノードからPDFファイルを取得します。
- 取得できたら、Notebook環境の[ダッシュボード（ファイル一覧画面）](./) でPDFファイルを開いて内容を確認します。

```
scp {VCノードIPアドレス}:ieicej3.2/UTF/readme.pdf .
```

In [ ]:
# VCノードから PDF ファイルをNotebook環境にコピー
!scp root@{ip_address}:ieicej3.2/UTF/readme.pdf .

### 他のtexファイルを使ってみる

- [学会の論文投稿スタイルファイル](https://texwiki.texjp.org/?学会スタイル等#ab53bdf2)

### 使用したVCノードを削除する

In [ ]:
# unit_group = vcp.get_ugroup('handson401')
unit_group.cleanup()
vcp.df_ugroups()

## VCノードを利用してWordPressやownCloud環境を構築してみよう

WordPress (CMS:コンテンツ管理システム) や、ownCloud (オンラインストレージ) のようなWebアプリケーション環境をVCノード上に構築します。

- WordPress
  * Docker公式ドキュメントページにある Docker Compose のサンプルを使って WordPress 環境を構築します。
  * [Quickstart: Compose and WordPress](https://docs.docker.com/compose/wordpress/)

- ownCloud
  * GitHubリポジトリに公開されている Docker Compose のサンプルを使って ownCloud 環境を構築します。
  * [ownCloud: Server](https://github.com/owncloud-docker/server)

### AWSにVCノードを作成

- WordPress, ownCloud どちらを構築する場合もVCノード作成手順は同じ
- 任意のインスタンスタイプを利用可（通常は `'small'` で十分）
- SSH公開鍵を設定する（起動後にSSHログインが必要なため）
- 固定IPアドレス `172.30.2.201` を割り当てる  
  (ハンズオン用NAT Proxyサーバに設定済みのIPアドレスを使う必要があるため)

In [ ]:
# アクセストークン入力
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
# VCP SDK 初期化
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

In [ ]:
# UnitGroup 作成
unit_group = vcp.create_ugroup('handson401')

In [ ]:
# spec 設定 (スポットインスタンスの場合は aws_spot を指定)
spec = vcp.get_spec('aws', 'small')

In [ ]:
# ssh公開鍵設定
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')
spec.set_ssh_pubkey(ssh_public_key)

In [ ]:
# 起動するVCノードに固定IPアドレス割当 (ハンズオン用NAT Proxyサーバに設定済みのIPアドレスを使う)
spec.ip_addresses = ['172.30.2.201']

In [ ]:
# spec内容確認
print(spec)

In [ ]:
# Unit 作成（VCノード起動）
unit = unit_group.create_unit('aws-server', spec)

In [ ]:
# VCノード情報確認
unit_group.df_nodes()

### VCノードにSSHログイン

NotebookのTerminalを開いてsshコマンドを実行してログインします。
```bash
ssh root@172.30.2.201
```

あるいは、[301-AWSスポットインスタンスの利用](./301-AWSスポットインスタンスの利用.ipynb) と同様にNotebookのセルにsshコマンドを記述して実行する方法もあります。

### Docker Compose 設定ファイルを記述

WordPress または ownCloud のどちらか一方を選択して実行してください。  
**!!注意!!** 今回のハンズオン環境では、1つのVCノードに WordPress と ownCloud の両方を同時に起動することはできません。

- **WordPress** の場合
  1. NotebookのTerminalから [Quickstart: Compose and WordPress](https://docs.docker.com/compose/wordpress/) に記載されている docker-compose.yml の内容をコピーし、  
VCノード上に `docker-compose.yml` ファイルを作成します。

----

- **ownCloud** の場合
  1. [ownCloudの公式ドキュメント](https://doc.owncloud.com/server/10.2/admin_manual/installation/docker/) の手順にしたがってVCノード上に `docker-compose.yml` ファイルを作成します。  
     以下のコマンドをVCノード上で実行してファイルを取得できます。

  ```bash
  wget -O docker-compose.yml \
      https://raw.githubusercontent.com/owncloud/docs/master/modules/admin_manual/examples/installation/docker/docker-compose.yml

  ```
  
  2. 環境変数を設定するために以下のコマンドをVCノード上で実行します。   
     ★今回のハンズオン環境では **`HTTP_PORT`** を **`8000`** に指定してください。★
  
  ```bash
  cat << EOF >| .env
  OWNCLOUD_VERSION=10.0
  OWNCLOUD_DOMAIN=localhost
  ADMIN_USERNAME=admin
  ADMIN_PASSWORD=admin
  HTTP_PORT=8000
  EOF
  ```

### Docker Compose 起動

VCノード上で `docker-compose.yml` ファイルを配置したディレクトリで `docker-compose up` コマンドを実行します。

```bash
docker-compose up -d
```

Docker Composeにより起動されたコンテナを確認します。

```bash
docker-compose ps
```

### ブラウザからアクセス

アクセス先URLのリンクをここで生成します。

In [ ]:
vcc_ctr = vcp.vcc_info()['host']
print("https://{}/".format(vcc_ctr.split(':')[0]))

### 使用したVCノードを削除する

In [ ]:
# unit_group = vcp.get_ugroup('handson401')
unit_group.cleanup()
vcp.df_ugroups()

## (参考) VCPで利用可能なDockerイメージを入手するには？

Dockerには開発元の Docker, Inc. により運営されているコンテナイメージの共有サービス [Docker Hub](https://hub.docker.com) があります。

Docker Hubでは、自分で作成したDockerイメージを保存したり、公開イメージとして配布したりすることが可能です。  
また、公開されているイメージを検索し、ダウンロードして自分のDocker環境で利用できます。

今回の演習課題でも Docker Hub から公開イメージをVCノード上に取得して実行しています。

演習で使用しているイメージ以外にも、基本OS (Ubuntu, CentOS) や一般的なプログラミング言語のランタイムなどを含む多くのソフトウェア製品が [Official Images](https://docs.docker.com/docker-hub/official_images/) として登録されており、これらは専門のチームによって品質とセキュリティの問題が発生するリスクを軽減するように対策されています。

![Docker architecture](https://docs.docker.com/engine/images/architecture.svg)
> [Docker architecture](https://docs.docker.com/engine/docker-overview/#docker-architecture)
> 
> Public Registry サービスとして Docker Hub があり、DockerクライアントはデフォルトでDocker Hub上のイメージを検索するように構成されています。
